<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [9]:
# brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "/home/kuangru/.local/bin/uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY"),"PATH": "/home/kuangru/.nvm/versions/node/v22.18.0/bin:" + os.environ["PATH"]}}
]

### Now create the MCPServerStdio for each

In [10]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=180) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=180) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [11]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [12]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [13]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=10)
display(Markdown(result.final_output))



The latest news on Amazon's financial performance is from their second quarter 2025 earnings report:

- Net sales increased 13% to $167.7 billion compared to $148.0 billion in Q2 2024.
- Sales in Amazon's online stores unit grew 11% year over year to $61.5 billion, surpassing Wall Street's projection of $59 billion.
- Seller services revenue rose 11% year over year to $40.3 billion, beating analyst expectations of $38.7 billion.

This strong quarterly performance highlights Amazon's continued growth and resilience. If you want, I can provide more detailed insights or additional news on Amazon.

### Look at the trace

https://platform.openai.com/traces

In [15]:
Janice_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Janice").reset(Janice_initial_strategy)

display(Markdown(await read_accounts_resource("Janice")))
display(Markdown(await read_strategy_resource("Janice")))

{"name": "janice", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-22 13:56:17", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [16]:
agent_name = "Janice"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [17]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Janice and your account is under your name, Janice.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "janice", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-22 13:56:17", 10000.0], ["2025-08-22 14:10:09", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so

### And to run our Trader

In [18]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions:

1. **Market Research**: 
   - Investigated recent market news, focusing on sectors with strong performance and notable stock movements in technology and retail sectors.

2. **Stock Price Checks**:
   - Checked current stock prices:
     - **Nvidia (NVDA)**: $175.00
     - **Intel (INTC)**: $23.50
     - **Home Depot (HD)**: $398.50
     - **AT&T (T)**: $29.44

3. **Trades Executed**:
   - **Bought 20 Shares of Nvidia (NVDA)** at $175.33:
     - **Rationale**: Strong earnings and demand in the tech sector, especially in semiconductors and AI.
   - **Bought 10 Shares of Home Depot (HD)** at $398.50:
     - **Rationale**: Better-than-expected earnings and robust consumer spending trends in retail.

4. **Current Portfolio Status**:
   - **Total Balance**: $2,508.45
   - **Holdings**:
     - Nvidia (NVDA): 20 shares
     - Home Depot (HD): 10 shares
   - **Total Portfolio Value**: $9,985.05
   - **Total Profit/Loss**: -$14.95 (after trading costs)

### Next Steps:
- Monitor the performance of NVDA and HD stocks over the next few hours for potential selling opportunities based on market movements and news.

### Then go and look at the trace

http://platform.openai.com/traces


In [19]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "janice", "balance": 2508.4467999999997, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"NVDA": 20, "HD": 10}, "transactions": [{"symbol": "NVDA", "quantity": 20, "price": 175.32996, "timestamp": "2025-08-22 14:15:37", "rationale": "Strong earnings and demand in the tech sector, specifically in semiconductors and AI."}, {"symbol": "HD", "quantity": 10, "price": 398.49539999999996, "timestamp": "2025-08-22 14:15:37", "rationale": "Better-than-expected earnings and solid consumer spending trends in the retail sector."}], "portfolio_value_time_series": [["2025-08-22 13:56:17", 10000.0], ["2025-08-22 14:10:09", 10000.0], ["2025-08-22 14:15:37", 9993.0008], ["2025-08-22 14:15:37", 9985.0468], ["2025-08-22 14:16:35", 9985.0468]], "total_portfolio_value": 9985.0468, "total_profit_loss": -14.953199999999924}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [2]:
from traders import Trader


In [3]:
trader = Trader("Janice")

In [4]:
await trader.run()

In [5]:
await read_accounts_resource("Janice")

'{"name": "janice", "balance": 664.9644999999998, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"NVDA": 15, "HD": 10, "PLTR": 10, "AAPL": 5}, "transactions": [{"symbol": "NVDA", "quantity": 20, "price": 175.32996, "timestamp": "2025-08-22 14:15:37", "rationale": "Strong earnings and demand in the tech sector, specifically in semiconductors and AI."}, {"symbol": "HD", "quantity": 10, "price": 398.49539999999996, "timestamp": "2025-08-22 14:15:37", "rationale": "Better-than-expected earnings and solid consumer spending trends in the retail sector."}, {"symbol": "PLTR", "quantity": 10, "price": 159.05748, "timestamp": "2025-08-24 13:14:04", "rationale": "High volatility and recent market uptrend, potential for quick gains."}, {"symbol": "NVDA", "quantity": -5, "price": 177.63402000000002, "timestamp": "2025-08-24 13:14:11", "rationale": "Realize profits and maintain a balance for new positions."}, {"symbol": "

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [6]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
